# Attempt at Template Matching
* 

In [4]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import os
import geopandas as gpd
from shapely.geometry import Polygon

f:\Python_Envs\BigPanel\Lib\site-packages\pyproj\__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_DATA (PROJ 9.1+) | PROJ_LIB (PROJ<9.1) or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


## Prepare Imagery

In [2]:
wd = r'S:\Users\stidjaco\R_files\BigPanel'
image = r'F:/naipPVtest.tif'
templateFolder = r'F:/PV_Templates_eCog'

In [3]:
# Read image using OpenCV
img = cv.imread(image)

# Load the main image
img = cv2.imread('path_to_your_image.tif')

# Load all templates from the directory
templates = []
for file in os.listdir(template_path):
    if file.endswith('.tif'):
        template = cv2.imread(os.path.join(template_path, file))
        templates.append(template)

# Initialize a mask to store matched areas
mask = np.zeros(img.shape[:2], dtype=np.uint8)

# Perform template matching for each template
for template in templates:
    result = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    loc = np.where(result >= 0.85)
    for pt in zip(*loc[::-1]):
        cv2.rectangle(mask, pt, (pt[0] + template.shape[1], pt[1] + template.shape[0]), 255, -1)

# Find contours from the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Convert contours to polygons
polygons = [Polygon(cnt.reshape(-1, 2)) for cnt in contours if len(cnt) >= 3]

# Create a GeoDataFrame from polygons
gdf = gpd.GeoDataFrame(geometry=polygons, crs='EPSG:4326')

# Save the vectorized polygons to a shapefile
output_shapefile = 'vectorized_output.shp'
gdf.to_file(output_shapefile)

print(f"Vectorized output saved to {output_shapefile}")